Lesson summary
To read multiple stocks into a single dataframe, you need to:

    Specify a set of dates using pandas.date_range
    Create an empty dataframe with dates as index
        This helps align stock data and orders it by trading date
    Read in a reference stock (here SPY) and drop non-trading days using pandas.DataFrame.dropna
    Incrementally join dataframes using pandas.DataFrame.join

Once you have multiple stocks, you can:

    Select a subset of stocks by ticker symbols
    Slice by row (dates) and column (symbols)
    Plot multiple stocks at once (still using pandas.DataFrame.plot)
    Carry out arithmetic operations across stocks, e.g. normalize by the first day's price


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def plot_selected(df, columns, start_index, end_index):

    df = df.ix[end_index:start_index,columns]
    ax = df.plot(title="SPY and IBM")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    plt.show()

def symbol_to_path(symbol, base_dir="data"):
    return os.path.join(base_dir, "{}.csv".format(str(symbol)))

def get_data(symbols):

    # Download csv - http://ichart.finance.yahoo.com/table.csv?d=6&e=1&f=2009&g=d&a=7&b=19&c=2004&ignore=.csv&s=GOOG
    
    df = pd.read_csv(symbol_to_path('SPY'), index_col='Date', parse_dates='TRUE', 
                     usecols=['Date','Adj Close'], na_values=['nan'])
    df = df.rename(columns={'Adj Close':'SPY'})
    df = df.ix['2010-12-31':'2010-01-02']
    
    for symbol in symbols:
        dftmp = pd.read_csv(symbol_to_path(symbol), index_col='Date', parse_dates='TRUE', 
                            usecols=['Date','Adj Close'], na_values=['nan'])
        dftmp = dftmp.rename(columns={'Adj Close':symbol})   
        df = df.join(dftmp,how='inner')
        if symbol == 'SPY':
            df=df.dropna(subset=['SPY'])
    return df

def test_run():

    # Choose stock symbols to read
    symbols = ['GOOG', 'IBM', 'GLD']
    
    # Get stock data
    df = get_data(symbols)
    return df
        
    # Slice and plot
    plot_selected(df, ['SPY', 'IBM'], '2010-03-01', '2010-04-01')

if __name__ == "__main__":
    dft = test_run()

In [4]:
dft.head(10)

,SPY,GOOG,IBM,GLD
Date,,,,
2010-12-31,113.503283,296.688813,130.298567,138.720001
2010-12-30,113.476206,299.131387,130.218665,137.029999
2010-12-29,113.656725,300.200325,130.085496,137.710007
2010-12-28,113.575493,299.161346,129.366352,137.220001
2010-12-27,113.413023,300.889622,129.037844,135.020004
2010-12-23,113.367890,301.813693,129.526156,134.660004
2010-12-22,113.530360,302.443073,129.579424,135.050003
2010-12-21,113.178343,301.234286,129.392986,135.320007
2010-12-20,112.465279,297.233273,128.300940,135.110001


In [2]:
# Normalized prices
dfn = dft / dft.ix[0]
dfn.head(10)

,SPY,GOOG,IBM,GLD
Date,,,,
2010-12-31,1.000000,1.000000,1.000000,1.000000
2010-12-30,0.999761,1.008233,0.999387,0.987817
2010-12-29,1.001352,1.011836,0.998365,0.992719
2010-12-28,1.000636,1.008334,0.992846,0.989187
2010-12-27,0.999205,1.014159,0.990324,0.973328
2010-12-23,0.998807,1.017274,0.994072,0.970732
2010-12-22,1.000239,1.019395,0.994481,0.973544
2010-12-21,0.997137,1.015321,0.993050,0.975490
2010-12-20,0.990855,1.001835,0.984669,0.973976


In [3]:
ax = dfn.plot(title="SPY and IBM")
ax.set_xlabel("Date")
ax.set_ylabel("Price")
plt.show()